In [ ]:
from copy import *
from random import *
from jogos import *
from utils import *
import string

from collections import namedtuple

stateBreakthru = namedtuple('EstadoBreakthru', 'to_move')

class EstadoBreakthru(stateBreakthru):

    def verificar_terminou(self):
        # 1 = White,  2 = Black
        if self.to_move == 1:
            for row in range(len(self.board)):
                if (self.board[row][0] == 'W'):
                    return 1
        elif self.to_move == 2:
            for row in range(len(self.board)):
                if (self.board[row][7] == 'B'):
                    return 1    
        return 0

    def __init__(self, to_move, n=8):
        self.board = [['.' for i in range(n)] for i in range(n)]
        for i in range(2):
            for j in range(8):
                self.board[i][j] = 'B'

        for i in range(6, 8):
            for j in range(8):
                self.board[i][j] = 'W'
        self.terminou = self.verificar_terminou()

    def moves(self, to_move):
        list_moves = set()
        for i in range(0,8):
            for j in range(0,8):
                if self.board[i][j] == 'B' and to_move == 2 and j+1 <= 7:
                    if self.board[i+1][j] == '.' and i+1 <= 7:
                       list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i+1,j)))
                    if (self.board[i+1][j+1] == 'W' or self.board[i+1][j+1] == '.') and i+1 <= 7 and j+1 <= 7:
                       list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i+1,j+1)))
                    if (self.board[i+1][j-1] == 'W' or self.board[i+1][j-1] == '.') and i+1 <=7 and j-1 >= 0:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i+1,j-1)))
                if self.board[i][j] == 'B' and to_move == 2 and j+1 == 8:
                    if self.board[i+1][j] == '.' and i+1 <= 7:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i+1,j)))
                    if (self.board[i+1][j-1] == 'W' or self.board[i+1][j-1] == '.') and i+1 <= 7:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i+1,j-1)))           

                if self.board[i][j] == 'W' and to_move == 1 and j+1 <= 7:
                    if self.board[i-1][j] == '.' and i-1 >= 0:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i-1,j)))
                    if (self.board[i-1][j+1] == 'B' or self.board[i-1][j+1] == '.') and i-1 >= 0 and j+1 <= 7:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i-1,j+1)))
                    if (self.board[i-1][j-1] == 'B' or self.board[i-1][j-1] == '.') and i-1 >= 0 and j-1 >= 0:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i-1,j-1)))
                if self.board[i][j] == 'W' and to_move == 1 and j+1 == 8:
                    if self.board[i-1][j] == '.' and i-1 >= 0:
                        list_moves.add("{pos1}-{pos2}".format(pos1=pos_convert(i,j), pos2=pos_convert(i-1,j)))
                    if (self.board[i-1][j-1] == 'B' or self.board[i-1][j-1] == '.') and i-1 >= 0:
                        list_moves.add("{pos1}-{pos2}".format(pos1=pos_convert(i,j), pos2=pos_convert(i-1,j-1)))
        
        return list_moves 
    
    def pos_convert(i, j):
        alfabeto = "abcdefgh"
        return alfabeto[j] + str(i + 1)

    def move_convert(move):
        numbers = []
        alfabeto = "abcdefgh"
        numbers.add(int(alfabeto.rfind(move[0])))
        numbers.add(int(move[1]) - 1)
        return numbers
        

## CLASS JOGO ##
class JogoBT_16(Game):
    """Play Breakthrough on an 8 x 8 board, with Max (first player) playing 'W' (1).
    """
    def __init__(self, to_move, n=8):
        self.initial = EstadoBreakthru(to_move=1, n=8)

    def actions(self, state):
        "Legal moves for B and W"
        #Apenas queremos ver as ações possíveis para o jogador atual (to_move)
        return state.moves(state.to_move)

    def result(self, state, move):
        if state.to_move == 1:
            index_start = state.move_convert(move[0:1])
            new_index = state.move_convert(move[3:4]) #talvez esteja errado
            self.board[index_start[0]][index_start[1]] = '.'
            self.board[new_index[0]][new_index[1]] = 'W'
            state.to_move = 2 #depois do branco jogar muda para o preto

        else:
            index_start = state.move_convert(move[0:1])
            new_index = state.move_convert(move[3:4]) #talvez esteja errado
            self.board[index_start[0]][index_start[1]] = '.'
            self.board[new_index[0]][new_index[1]] = 'B'
            state.to_move = 1 #depois do preto jogar muda para o branco


    def terminal_test(self, state):
        "A state is terminal if the whites reached line 8 or some black reached line 1."
        return state.terminou == 1

    def display(self, state):
        symbols = "abcdefgh"
        print("-"*17)
        for i in range(len(state.board)):
            print("{num}|".format(num=8-i) + ' '.join(state.board[i]))
        print("-+" + "-"*15)
        print(" |" + ' '.join(i for i in symbols))

        if state.to_move == 1:
            print("--NEXT PLAYER: W")
        elif state.to_move == 2:
            print("--NEXT PLAYER: B")

    def executa(self, estado, listaJogadas):
        "executa varias jogadas sobre um estado dado"
        "devolve o estado final "
        s = estado
        for j in listaJogadas:
            s = self.result(s, j)
        return s